In [ ]:
#파이썬 지도 생성 라이브러리 활용
pip install folium

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("/Users/yoonjihun/Downloads/Churn.csv", index_col=0)

In [2]:
df.rename(lambda x: x.lower().strip().replace(' ', '_'), axis='columns', inplace=True)

In [ ]:
df.loc[df['customer_status'] == 'Churned', 'monthly_avg_ld_charge'] = df['total_long_distance_charges'] / df['tenure_in_months']

In [ ]:
#장거리 추가 요금 지불 요금 고객의 지역 분포를 알아보기 위해 지도 그리기
third_quartile = df['monthly_avg_ld_charge'].quantile(0.75)
df_up = df[df['monthly_avg_ld_charge'] > third_quartile]
df_down = df[df['monthly_avg_ld_charge'] <= third_quartile]

In [ ]:
import folium

m = folium.Map(location=[36.19745482180888, -119.75668367414455], zoom_start=7)

for idx, row in df_up.iterrows():
    lat, lng = row.latitude, row.longitude
    folium.CircleMarker(location = [lat, lng],
                     radius = 5,
                     color='red',
                     fill = True,
                     fill_color='red').add_to(m)

for idx, row in df_down.iterrows():
    lat, lng = row.latitude, row.longitude
    folium.CircleMarker(location = [lat, lng],
                     radius = 5,
                     color='blue',
                     fill = True,
                     fill_color='blue').add_to(m)
m

마커로 표시하면 구별이 되지 않아서 클러스터와 히트맵 등의 방법을 찾아보았지만 두 고객집단을 구별하여 표시하기는 불가

토탈 장거리 추가 차지 / 티뉴어 기간으로 월평균 추가 장거리 요금과 지역과의 상관관계

확인해보려 했으나 추가 장거리 요금과 이탈율에 대한 상관관계는 발견되지 않음

In [ ]:
df['total_revenue'].mean()

In [ ]:
# 충성고객과 높은 리스크로 이탈한 고객들의 이탈률 지도로 표현
df_high_value = df[(df['total_revenue'] > 1.5*df['total_revenue'].mean()) & (df['customer_status'] == 'Stayed')]
df_risk_churn = df[(df['customer_status'] == 'Churned') & (df['tenure_in_months'] < 3)]

In [ ]:
import folium

m = folium.Map(location=[36.19745482180888, -119.75668367414455], zoom_start=7)

for idx, row in df_high_value.iterrows():
    lat, lng = row.latitude, row.longitude
    folium.CircleMarker(location = [lat, lng],
                     radius = 3,
                     color='red',
                     fill = True,
                     fill_color='red').add_to(m)

for idx, row in df_risk_churn.iterrows():
    lat, lng = row.latitude, row.longitude
    folium.CircleMarker(location = [lat, lng],
                     radius = 4,
                     color='blue',
                     fill = True,
                     fill_color='blue').add_to(m)
m

그러나 우수고객과 단기간 내 이탈한 위험고객 사이의 지역분포도를 보면 도심(LA) 지역에서 이탈자가 높은 것으로 보임
그러나 큰 경향성은 파악하지 못함
이를 확인하기 위해 사람이 많은 지역을 살펴보기로 함

In [3]:
top_5_cities = df['city'].value_counts().head(5)
print(top_5_cities)

Los Angeles      293
San Diego        285
San Jose         112
Sacramento       108
San Francisco    104
Name: city, dtype: int64


In [4]:
#인구 상위 5개 지역의 이탈여부 지도 표현
df_city_churn = df[(df['customer_status'] == 'Churned') & (df['city'].isin(['Los Angeles', 'San Diego', 'San Jose', 'Sacramento', 'San Francisco']))]
df_city_stay = df[(df['customer_status'] == 'Stayed') & (df['city'].isin(['Los Angeles', 'San Diego', 'San Jose', 'Sacramento', 'San Francisco']))]

In [5]:
df_city_stay[df_city_stay['customer_status'] == 'Stayed'].info()

<class 'pandas.core.frame.DataFrame'>
Index: 513 entries, 0048-PIHNL to 9979-RGMZT
Data columns (total 37 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   gender                             513 non-null    object 
 1   age                                513 non-null    int64  
 2   married                            513 non-null    object 
 3   number_of_dependents               513 non-null    int64  
 4   city                               513 non-null    object 
 5   zip_code                           513 non-null    int64  
 6   latitude                           513 non-null    float64
 7   longitude                          513 non-null    float64
 8   number_of_referrals                513 non-null    int64  
 9   tenure_in_months                   513 non-null    int64  
 10  offer                              513 non-null    object 
 11  phone_service                      513 non-null

In [ ]:
import folium

m = folium.Map(location=[36.19745482180888, -119.75668367414455], zoom_start=7)

for idx, row in df_city_churn.iterrows():
    lat, lng = row.latitude, row.longitude
    folium.CircleMarker(location = [lat, lng],
                     radius = 5,
                     color='red',
                     fill = True,
                     fill_color='red').add_to(m)

for idx, row in df_city_stay.iterrows():
    lat, lng = row.latitude, row.longitude
    folium.CircleMarker(location = [lat, lng],
                     radius = 4,
                     color='blue',
                     fill = True,
                     fill_color='blue').add_to(m)
m